# Implementación de PCA en NumPy

## Objetivos
* Implementación de PCA en NumPy paso a paso
* Comparación de resultados con Scikit-learn

In [46]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

## Implementación

1. Dado un dataset $X \in \mathbb{R}^{n, d}$, con $n$ muestras y $d$ features, queremos reducir sus dimensiones a $m$. Para ello, el primer paso es centrar el dataset (Hint: usen np.mean)

In [34]:
# cantidad de muestras
n = 100

# dimensión inicial
m = 3

# dimensión reducida
d = 2


# Muestra random
X_1 = np.random.normal(loc=[-5, -5, -5], scale=[1, 10, 5], size=(int(n/4),m))
X_2 = np.random.normal(loc=[1, 1, 1], scale=[1, 10, 5], size=(int(n/4),m))
X_3 = np.random.normal(loc=[-10, 10, 10], scale=[1, 10, 5], size=(int(n/4),m))
X_4 = np.random.normal(loc=[1, -2, 3], scale=[2, 2, 2], size=(int(n/4),m))

X = np.concatenate((X_1, X_2, X_3, X_4))
print("X shape:")
print(X.shape)
print("X head:")
print(X[:5,:])

X shape:
(100, 3)
X head:
[[ -3.87333239 -14.72777593   1.06263634]
 [ -6.35196476   7.00345829 -13.48017221]
 [ -5.80403836 -10.57548385 -14.36992022]
 [ -5.09108229   1.31845882  -9.71189046]
 [ -5.59589367 -11.98410246  -7.34810483]]


In [35]:
X_mean = X.mean(axis=0)
X_centered = X - X_mean

2. Obtener la matriz de covarianza de $X^T$, revisar en la teoría por qué utilizamos la transpuesta. Buscar en la documentación de NumPy qué funciones se pueden utilizar.

In [36]:
cov_mtx = np.cov(X_centered.T) # Nos interesa saber cuál es la covarianza de las features, no de las realizaciones.
cov_mtx.shape

(3, 3)

3. Calcular los autovalores y autovectores de la matriz de covarianza. Revisar la documentación de NumPy.

In [37]:
eig_w, eig_v = np.linalg.eig(cov_mtx)
print(eig_w)
print(eig_v)

[119.7731195   16.67964837  44.54499189]
[[ 0.17702684 -0.96788551 -0.17849129]
 [-0.94081145 -0.11315804 -0.31948249]
 [-0.28902475 -0.22448363  0.93062978]]


4. Ordernar los autovectores en el sentido de los autovalores decrecientes, revisar la teoría de ser necesario.

In [38]:
arg_sort = np.argsort(eig_w)[::-1]
eig_w = eig_w[arg_sort]
eig_v = eig_v[:,arg_sort]
print(eig_w)
print(eig_v)

[119.7731195   44.54499189  16.67964837]
[[ 0.17702684 -0.17849129 -0.96788551]
 [-0.94081145 -0.31948249 -0.11315804]
 [-0.28902475  0.93062978 -0.22448363]]


5. Proyectar el dataset centrado sobre los $m$ autovectores más relevantes (Hint: usen np.dot).

In [39]:
proj = X.dot(eig_v[:,:d])
print(proj.shape)

(100, 2)


6. Consolidar los pasos anteriores en una función o clase PCA.

In [41]:
def custom_pca(X, num_comp=2):
    X = X - X.mean(axis=0)
    cov_mtx = np.cov(X.T)

    eig_w, eig_v = np.linalg.eig(cov_mtx)

    arg_sort = np.argsort(eig_w)[::-1]
    eig_w = eig_w[arg_sort]
    eig_v = eig_v[:,arg_sort]

    proj = X.dot(eig_v[:,:num_comp])
    proj_v = eig_v[:num_comp]

    return proj_v, proj

7. Comparar los resultados obtenidos con el modelo de PCA implementado en Scikit-learn ([ver documentación](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html)). Tomar como dataset:

$X=\begin{bmatrix}
0.8 & 0.7\\
0.1 & -0.1
\end{bmatrix}$

Se debe reducir a un componente. Verificar los resultados con np.testing.assert_allclose

In [56]:
X_test = np.array([[0.8, 0.7], [0.1, -0.1]])
X_scaled = StandardScaler(with_std=False).fit_transform(X_test)
X_scaled

array([[ 0.35,  0.4 ],
       [-0.35, -0.4 ]])

In [57]:
pca_skl = PCA(n_components=1)
pca_skl.fit(X_scaled)
X_pca_skl = pca_skl.transform(X_scaled)
X_pca_skl

array([[-0.53150729],
       [ 0.53150729]])

In [61]:
eig_v, X_pca_cust = custom_pca(X_test, 1)
X_pca_cust

array([[-0.53150729],
       [ 0.53150729]])

In [62]:
np.testing.assert_allclose(X_pca_skl, X_pca_cust)